# Install the required packages

In [1]:
import warnings
warnings.simplefilter('ignore')
import keras
import tensorflow as tf
print(tf.__version__)

2.3.0


In [2]:
import cv2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array
from keras.models import Model, load_model, Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.metrics import Accuracy
from sklearn.model_selection import train_test_split
from keras import utils
import imutils
import numpy as np 
import matplotlib.pyplot as plt 


# Data Augmentation

In [3]:
#train_datagen = ImageDataGenerator(rotation_range=40,
#                                   width_shift_range=0.2,
#                                  height_shift_range=0.2,
#                                   rescale=1.0/255,
#                                   shear_range=0.2,
#                                   zoom_range=0.2,
#                                   horizontal_flip=True,
#                                   fill_mode='nearest')

#because already we have taken another dataset which is augmented so need all this features

train_datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1.0/255)


# Read the train directory images

In [ ]:
# import io
# from PIL import Image
# import glob

# image_list = []
# for filename in glob.glob(r'E:\AI-ML PYTHON\Project-7\Mask dataset\dest_folder\train/*.jpg'): #assuming gif
#     im=Image.open(filename)
#     image_list.append(im)
    
# result = ''.join([s.encode('ascii','ignore') for s in image_list])

# # This portion is part of my test code
# byteImgIO = io.BytesIO()
# byteImg = Image.open(result)
# byteImg.save(byteImgIO, "PNG")
# byteImgIO.seek(0)
# byteImg = byteImgIO.read()


# # Non test code
# dataBytesIO = io.BytesIO(byteImg)
# Image.open(dataBytesIO)


In [4]:
batch_size = 30

train_data = train_datagen.flow_from_directory(r"E:\AI-ML PYTHON\Project-7\Mask dataset\data\train",
                                               target_size=(100,100),
                                               batch_size=batch_size,
                                               class_mode='binary')


Found 1376 images belonging to 2 classes.


# Read the test directory images

In [5]:
test_data = test_datagen.flow_from_directory(r"E:\AI-ML PYTHON\Project-7\Mask dataset\data\test",
                                            target_size = (100,100),
                                            batch_size = batch_size,
                                            class_mode = 'binary')

Found 194 images belonging to 2 classes.


# Building The Model

In [ ]:
# i = Input(shape=(100,100,3))

# x = Conv2D(100, (3, 3), activation='relu', padding='same')(i)
# x = BatchNormalization()(x)
# x = Conv2D(100, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2, 2))(x)
# # x = Dropout(0.2)(x)
# x = Conv2D(200, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(200, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2, 2))(x)
# # x = Dropout(0.2)(x)
# x = Conv2D(300, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = Conv2D(300, (3, 3), activation='relu', padding='same')(x)
# x = BatchNormalization()(x)
# x = MaxPooling2D((2, 2))(x)
# # x = Dropout(0.2)(x)

# # x = GlobalMaxPooling2D()(x)
# x = Flatten()(x)
# x = Dropout(0.5)(x)
# x = Dense(1024, activation='relu')(x)
# x = Dropout(0.3)(x)
# x = Dense(2, activation='softmax')(x)

# model = Model(i,x)  


In [6]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=(100,100,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories


# compile the model

In [7]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [8]:
checkpoint = ModelCheckpoint('model2-{epoch:03d}.model',monitor='val_loss',
                             verbose=0,save_best_only=True,mode='auto')
callbacks_list = [checkpoint]

In [9]:
histroy=model.fit( train_data,steps_per_epoch=100//batch_size,epochs=15,
                            callbacks=callbacks_list,
                             validation_data=test_data,validation_steps=70//batch_size)

Epoch 1/15
3/3 [==============================] - ETA: 0s - loss: 1.3013 - accuracy: 0.4556WARNING:tensorflow:From D:\ANACONDA\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model2-001.model\assets
3/3 [==============================] - 8s 3s/step - loss: 1.3013 - accuracy: 0.4556 - val_loss: 0.6952 - val_accuracy: 0.5000
Epoch 2/15
3/3 [==============================] - 7s 2s/step - loss: 0.6940 - accuracy: 0.4222 - val_loss: 0.6932 - val_accuracy: 0.4667
Epoch 3/15
3/3 [==============================] - 6s 2s/step - loss: 0.6932 - accuracy: 0.5111 - val_loss: 0.6931 - val_accu

In [10]:
model.save('mask_detection.h5')

In [11]:
model = tf.keras.models.load_model('mask_detection.h5')

In [12]:
face_clsfr = cv2.CascadeClassifier(r"E:\AI-ML PYTHON\Project-7\Mask dataset/haarcascade_frontalface_default - Govindaraj V.xml")


In [13]:
results={0:'Mask',1:'No mask'}
GR_dict={0:(0,255,0),1:(0,0,255)}

rect_size = 4


In [ ]:
capture = cv2.VideoCapture(0)

while True:
    (ret,img) = capture.read()
    img = cv2.flip(img,1,1)
    
    rerect_size = cv2.resize(img, (img.shape[1] // rect_size, img.shape[0] // rect_size))
    faces = face_clsfr.detectMultiScale(rerect_size)
    
    for f in faces:
        (x,y,w,h) = [i*rect_size for i in f]
        
        face_img = img[y:y+h, x:x+w]
        rerect_size = cv2.resize(face_img,(100,100))
        normalized=rerect_size/255.0
        reshaped=np.reshape(normalized,(1,100,100,3))
        reshaped=np.vstack([reshaped])
        result=model.predict(reshaped)
        
        label=np.argmax(result,axis=1)[0]
        
        
        
        
        cv2.rectangle(img,(x,y), (x+w, y+h),GR_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),GR_dict[label],-1)
        cv2.putText(img, results[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)

        

    cv2.imshow('result',img)
    if cv2.waitKey(2)==27:
            break
capture.release() 
cv2.destroyAllWindows()
    


In [ ]:
# source=cv2.VideoCapture(0)

# labels_dict={0:'MASK',1:'NO MASK'}
# color_dict={0:(0,255,0),1:(0,0,255)}

In [ ]:
# while(True):

#     ret,img=source.read()
# #    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
#     faces=face_clsfr.detectMultiScale(img,1.3,5)  

#     for x,y,w,h in faces:
    
#         face_img=img[y:y+w,x:x+w]
#         resized=cv2.resize(face_img,(100,100))
#         normalized=resized/255.0
#         reshaped=np.reshape(normalized,(1,100,100,3))
#         result=model.predict(reshaped)

#         label=np.argmax(result,axis=1)[0]
      
#         cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
#         cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
#         cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
#     cv2.imshow('LIVE',img)
#     key=cv2.waitKey(1)
    
#     if(key==27):
#         break
        
# cv2.destroyAllWindows()
# source.release()